In [38]:
import numpy as np

## * Functions for spreading or packing tensors in lower dimension

In [39]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1 #출력물의 높이
    out_w = (W + 2*pad - filter_w)//stride + 1 #출력물의 너비

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [40]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [41]:
x1 = np.random.rand(1,3,7,7)
x1.shape

(1, 3, 7, 7)

In [42]:
col1 = im2col(x1, 5, 5, stride=1, pad=0)
col1.shape

(9, 75)

<br><hr><br>

## * Classes implementing ReLU & Softmax layers

In [45]:
class ReLU : 
    def __init__(self) : 
        self.mask = None
        
    def forward(self, x) : 
        self.mask = (x<=0) #numpy 배열 x와 같은 shape을 가진 배열을 만들고, 0 이상인 셀은 True, 0 미만인 셀은 False인 배열을 반환
        out = x.copy() 
        out[self.mask] = 0 #numpy 배열인 x 중 0 이하인 값은 0으로 치환
        return out
    
    def backward(self, dout) : 
        dout[self.mask] = 0 #numpy 배열 dout 중 0 이하인 값은 0으로 치환
        dx = dout
        return dx

In [47]:
#3-5) softmax_function
def softmax_function(x) : 
    return np.exp(x-max(x))/np.sum(np.exp(x-max(x)))

In [48]:
#4-4) CEE (cross_entropy_error, 교차엔트로피오차)
def CEE(y_p, y_t) : #y_p: predicted vector, y_t: actual_vector
    if y.ndim == 1 : 
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    delta = 1e-7
    return -1/y_p.shape[0]*np.sum(y_t*np.log(y_p + delta)) #y_p가 0이 되어 log 계산시 -inf가 나오는 현상 방지

In [46]:
class SofwmaxWithLoss : 
    def __init__(self) : 
        self.loss = None #최종 결과물인 loss값
        self.y = None #predicted Y
        self.t = None #actual Y
        
    
    def forward(self, x, t) : 
        self.t = t
        self.y = softmax_function(x)
        self.loss = CEE(self.y, self.t)
        return self.loss
    
    
    def backward(self, dout=1) : 
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

<br><hr><br>

## * Classes implementing Convolution & Pooling layers

In [43]:
class Convolution : 
    def __init__(self, W, b, stride=1, pad=0) : 
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        
    def forward(self, x) : 
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T 
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_W, -1).transpose(0, 3, 1, 2)
        
        return out
    
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx


In [44]:
class Pooling : 
    def __init__(self, pool_h, pool_w, stride=1, pad=0) : 
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None
    
    
    def forward(self, x) : 
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        #전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        arg_max = np.argmax(col, axis=1)
        
        #max pooling
        out = np.max(col, axis=1) 
        
        #packing
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        self.x = x
        self.arg_max = arg_max

        return out
    

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

<br><hr><br>

In [ ]:
class SimpleConvNet : 
    def __init__(self, input_dim=(1,28,28), conv_param={"filter_num":30, "filter_size":5, "pad":0, "stride":1},
                        hidden_size=100, output_size=10, weight_init_std=0.01) : 

        #매개변수 설정
        filter_num = conv_param["filter_num"]
        filter_size = conv_param["filter_size"]
        filter_pad = conv_param["pad"]
        filter_stride = conv_param["stride"]
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        #각 레이어의 가중치 매개변수 설정
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params["b1"] = np.zeros(filter_num)
        self.params["W2"] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params["b2"] = np.zeros(hidden_size) 
        self.params["W3"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b3"] = np.zeros(output_size) 
        
        #각 레이어 초기화
        self.layers = OrderDict()
        self.layers["Conv1"] = Convolution(self.params["W1"], self.params["b1"], conv_param["stride"], conv_param["pad"])
        self.layers["ReLU1"] = ReLU()
        self.layers["Pool1"] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers["Affine1"] = Affine(self.params["W2"], self.params["b2"])
        self.layers["ReLU2"] = ReLU()
        self.layers["Affine2"] = Affine(self.params["W3"], self.params["b3"])
        
        self.last_layer = SoftmaxWithLoss()
        
    
    def predict(self, x) : 
        for layer in self.layers.values() : 
            x = layer.forward(x)            
        return x 
    
    
    def loss(self, x, y) : 
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    
    def gradient(self, x, t) : 
        #순전파 (순전파하면서 가중치 매개변수들 갱신)
        self.loss(x, t) 
        
        #역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers :
            dout = layer.backward(dout)
            
        #결과 저장
        grads = {}
        grads["W1"] = self.layers["Conv1"].dW
        grads["b1"] = self.layers["Conv1"].db
        grads["W2"] = self.layers["Affine1"].dW
        grads["b2"] = self.layers["Affine1"].db
        grads["W3"] = self.layers["Affine2"].dW
        grads["b3"] = self.layers["Affine2"].db
        
        return grads